In [15]:
import os
import dotenv
import openai
import json
dotenv.load_dotenv()
client = openai.OpenAI()

In [2]:
def show_json(obj):
    display(json.loads(obj.model_dump_json()))
   

In [3]:
file = client.files.create(
    file=open(
        "test.xlsx",
        "rb",
    ),
    purpose="assistants",
)

In [4]:
assistant = client.beta.assistants.create(
    name="Analyste",
    instructions="""Tu es un expert pour analyser des documents excel tu le fais de manière simple et efficace.""",
    model="gpt-4-1106-preview",
    file_ids=[file.id],
    tools=[{"type": "code_interpreter"}, {"type": "retrieval"}],
    
)
MY_ASSISTANT_ID = assistant.id  # or a hard-coded ID like "asst-..."
show_json(assistant)

{'id': 'asst_RuYu0xbZH5P4Mx7LGNDI8mTS',
 'created_at': 1700755233,
 'description': None,
 'file_ids': ['file-uCuPD3CrpKCQEwp9H0nbQz5o'],
 'instructions': 'Tu es un expert pour analyser des documents excel tu le fais de manière simple et efficace.',
 'metadata': {},
 'model': 'gpt-4-1106-preview',
 'name': 'Analyste',
 'object': 'assistant',
 'tools': [{'type': 'code_interpreter'}, {'type': 'retrieval'}]}

In [5]:

def submit_message(assistant_id, thread, user_message):
    client.beta.threads.messages.create(
        thread_id=thread.id, role="user", content=user_message
    )
    return client.beta.threads.runs.create(
        thread_id=thread.id,
        assistant_id=assistant_id,
    )


def get_response(thread):
    return client.beta.threads.messages.list(thread_id=thread.id, order="asc")

def create_thread_and_run(user_input):
    thread = client.beta.threads.create()
    run = submit_message(MY_ASSISTANT_ID, thread, user_input)
    return thread, run

def append_message_and_create_run(thread, user_input):
    client.beta.threads.messages.create(
        thread_id=thread.id, role="user", content=user_input
    )
    return client.beta.threads.runs.create(
        thread_id=thread.id,
        assistant_id=MY_ASSISTANT_ID,
    )

import time

# Pretty printing helper
def pretty_print(messages):
    print("# Messages")
    for m in messages:
        print(f"{m.role}: {m.content[0].text.value}")
    print()


# Waiting in a loop
def wait_on_run(run, thread):
    while run.status == "queued" or run.status == "in_progress":
        run = client.beta.threads.runs.retrieve(
            thread_id=thread.id,
            run_id=run.id,
        )
        time.sleep(0.5)
    return run    

In [6]:
thread1, run1 = create_thread_and_run("""Fais moi une liste de tout ce que l'on peut savoir sur la cuma avec le numéro de siret 42040023600014
et renvoie moi les valeurs associées""")

In [8]:
run1 = wait_on_run(run1, thread1)

KeyboardInterrupt: 

In [ ]:
pretty_print(get_response(thread1))